In [1]:
import os
from azureml.core import Workspace
from azureml.core import Environment
from azureml.core import ScriptRunConfig, Experiment
from azureml.widgets import RunDetails
from azureml.core import Model

In [2]:
# AMLワークスペースへの接続
ws = Workspace.from_config()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code SC6H7XM77 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
# training script
source_dir = "train/resnet"
script_name = "train_resnet.py"

# environment file
environment_file = os.path.join("train", "resnet", "train_resnet_env.yml")

# azure ml settings
environment_name = "pl-env-image"
experiment_name = "resnet-cifar10"
compute_name = "gpu-instance01"

In [4]:
# 学習環境作成、初回のみ長時間
env = Environment.from_conda_specification(environment_name, environment_file)

env.docker.enabled = True
env.docker.base_image = (
    "mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04"
)

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [5]:
# 学習設定
src = ScriptRunConfig(
    source_directory=source_dir,
    script=script_name,
    arguments=["--batch_size", 512, "--max_epochs", 7, "--gpus", 1],
    compute_target=compute_name,
    environment=env,
)

In [6]:
# 実行
run = Experiment(ws, experiment_name).submit(src)
run.wait_for_completion(show_output=True)

Submitting /home/shunta_ito/notebooks/workspace/azureml-pl-sample/train directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


SnapshotException: SnapshotException:
	Message: ====================================================================

While attempting to take snapshot of /home/shunta_ito/notebooks/workspace/azureml-pl-sample/train
Your total snapshot size exceeds the limit of 300.0 MB.
Please see http://aka.ms/aml-largefiles on how to work with large files.

====================================================================


	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "====================================================================\n\nWhile attempting to take snapshot of /home/shunta_ito/notebooks/workspace/azureml-pl-sample/train\nYour total snapshot size exceeds the limit of 300.0 MB.\nPlease see http://aka.ms/aml-largefiles on how to work with large files.\n\n====================================================================\n\n"
    }
}

In [ ]:
# モデル登録
run.register_model(
    model_name="resnet-cifar10-model",
    model_path=os.path.join('outputs', 'model.onnx'),
    model_framework=Model.Framework.ONNX,
)